## Part 1: Preprocessing

In [437]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [438]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [439]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [440]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime',
             'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
             'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [441]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [442]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_cat_cols = X_train.select_dtypes(include='object').columns.tolist()
X_encoder = OneHotEncoder(drop='if_binary', sparse_output=False, handle_unknown='ignore')
X_encoder.fit(X_train[X_cat_cols])

X_train_encoded = pd.DataFrame(X_encoder.transform(X_train[X_cat_cols]), index=X_train.index)
X_test_encoded = pd.DataFrame(X_encoder.transform(X_test[X_cat_cols]), index=X_test.index)

X_train = X_train.drop(columns=X_cat_cols)
X_test = X_test.drop(columns=X_cat_cols)
X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_test = pd.concat([X_test, X_test_encoded], axis=1)

# Check the count of each value in the 'OverTime' column
attrition_df["OverTime"].value_counts()

OverTime
No     1054
Yes     416
Name: count, dtype: int64

In [443]:
# Ensure all column names are strings
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the testing data
X_test_scaled = scaler.transform(X_test)

In [444]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
dept_encoder.fit(y_train[["Department"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept = dept_encoder.transform(y_train[["Department"]])
y_test_dept = dept_encoder.transform(y_test[["Department"]])

# Print encoded department values to visualize the result
print(y_train_dept)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [445]:
# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
attr_encoder.fit(y_train[["Attrition"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr = attr_encoder.transform(y_train[["Attrition"]])
y_test_attr = attr_encoder.transform(y_test[["Attrition"]])

## Part 2: Create, Compile, and Train the Model

In [446]:
# Find the number of columns in the X training data.
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = Input(shape=(input_dim,), name='input')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer1)


In [447]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_branch = layers.Dense(64, activation='relu', name='department_hidden')(shared_layer2)

# Create the output layer
dept_output = layers.Dense(y_train_dept.shape[1], activation='relu', name='department_output')(dept_branch)

In [448]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_branch = layers.Dense(64, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attr_output = layers.Dense(units=y_train_attr.shape[1], activation='relu', name='attrition_output')(attr_branch)

In [449]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attr_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department_output': 'categorical_crossentropy',
        'attrition_output': 'categorical_crossentropy'
    },
    metrics={
        'department_output': 'accuracy',
        'attrition_output': 'accuracy'
    }
)

# Summarize the model
model.summary()


Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 10)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared1 (Dense)     │ (None, 64)        │        704 │ input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared2 (Dense)     │ (None, 128)       │      8,320 │ shared1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 64)        │      8,256 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 64)        │      8,256 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │        195 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │        130 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 25,861 (101.02 KB)

 Trainable params: 25,861 (101.02 KB)

 Non-trainable params: 0 (0.00 B)

In [450]:
# Train the model
# Train the model using the preprocessed data
history = model.fit(
    X_train_scaled,
    {'department_output': y_train_dept, 'attrition_output': y_train_attr},
    epochs=100,
    batch_size=32,
    validation_split=0.2,  # Optional, gives a validation curve
    verbose=1
)


Epoch 1/100


28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - attrition_output_accuracy: 0.8542 - attrition_output_loss: nan - department_output_accuracy: 0.5914 - department_output_loss: 5.1231 - loss: nan - val_attrition_output_accuracy: 0.8281 - val_attrition_output_loss: nan - val_department_output_accuracy: 0.6335 - val_department_output_loss: 5.9029 - val_loss: nan
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8441 - attrition_output_loss: nan - department_output_accuracy: 0.6621 - department_output_loss: 5.4472 - loss: nan - val_attrition_output_accuracy: 0.8281 - val_attrition_output_loss: nan - val_department_output_accuracy: 0.6335 - val_department_output_loss: 5.9029 - val_loss: nan
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8516 - attrition_output_loss: nan - department_output_accuracy: 0.6739 - department_output_loss: 5.2578 - loss: nan - val_attrition_output_accuracy: 0.8281 - val_attrition_output_loss: nan - val_depa

In [451]:
# Evaluate the model with the testing data
model.evaluate(X_test_scaled, {'department_output': y_test_dept, 'attrition_output': y_test_attr}, verbose=1)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.7898 - attrition_output_loss: nan - department_output_accuracy: 0.6296 - department_output_loss: 5.9389 - loss: nan 


[nan, 5.582569122314453, nan, 0.8152173757553101, 0.6413043737411499]

In [452]:
# Print the accuracy for both department and attrition
print("Department Accuracy: ", np.mean(predicted_dept == np.argmax(y_test_dept, axis=1)))
print("Attrition Accuracy: ", np.mean(predicted_attr == np.argmax(y_test_attr, axis=1)))
# Make predictions on the test data
predictions = model.predict(X_test_scaled)
# Get the predicted department and attrition values
predicted_dept = np.argmax(predictions[0], axis=1)
predicted_attr = np.argmax(predictions[1], axis=1)


Department Accuracy:  0.03804347826086957
Attrition Accuracy:  0.8152173913043478
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric to use on this data. The reasoning is the imbalance in the dataset regarding overtime. There is a bias towards No with 1054 vs Yes with 416.
2. I chose Relu since it is generally more computationally efficient (faster), but this is not ideal for the output layers since they are multi class/binary which is where softmax/sigmoid may be more pertinent.
3. Addressing imbalance in the data classes, hyperparameter tuning, and feature engineering could improve this model. This could get ride of biases, increase efficiency, and capture unique patterns.